In [28]:
%load_ext autoreload
%autoreload 2
from knowme.ingest import NotionIngestor
from knowme.embedder import ChromaEmbedder

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
ingestor = NotionIngestor(
    folder_path="/Users/abhinavkashyap/abhi/projects/knowme/abhinav_notion"
)
texts, metadatas = ingestor.ingest()

In [29]:
splitter

['# Abhinav Ramesh Kashyap\n\n![abhinav_pic.jpeg](Abhinav%20Ramesh%20Kashyap%202a5b5c4c70f240359dc297eaa025b430/abhinav_pic.jpeg)\n\n[https://notaku.so/iframes/clickable-image/eyJpbWFnZVVybCI6Imh0dHBzOi8vZW5jcnlwdGVkLXRibjAuZ3N0YXRpYy5jb20vaW1hZ2VzP3E9dGJuOkFOZDlHY1FPWnRzS0VJMTFOR3FMVk1ObFZ2WE5fUTZUeVhNUXVPZ2Q1U0IwV0Y2TzdRJnMiLCJsaW5rSHJlZiI6Imh0dHBzOi8vc2Nob2xhci5nb29nbGUuY29tL2NpdGF0aW9ucz91c2VyPXVjM3UwWkFBQUFBSiZobD1lbiJ9](https://notaku.so/iframes/clickable-image/eyJpbWFnZVVybCI6Imh0dHBzOi8vZW5jcnlwdGVkLXRibjAuZ3N0YXRpYy5jb20vaW1hZ2VzP3E9dGJuOkFOZDlHY1FPWnRzS0VJMTFOR3FMVk1ObFZ2WE5fUTZUeVhNUXVPZ2Q1U0IwV0Y2TzdRJnMiLCJsaW5rSHJlZiI6Imh0dHBzOi8vc2Nob2xhci5nb29nbGUuY29tL2NpdGF0aW9ucz91c2VyPXVjM3UwWkFBQUFBSiZobD1lbiJ9)\n\n[https://notaku.so/iframes/clickable-image/eyJpbWFnZVVybCI6Imh0dHBzOi8vZW5jcnlwdGVkLXRibjAuZ3N0YXRpYy5jb20vaW1hZ2VzP3E9dGJuOkFOZDlHY1RBRDN2N2hqS1BUV2hZQ1loXzhnRWU2ekJUVGJzdmFTdFZPWERVcnpTNGRnJnMiLCJsaW5rSHJlZiI6Imh0dHBzOi8vd3d3LnNlbWFudGljc2Nob2xhci5vcmcvbWUvcmVzZWFyY2gi